<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/face_detection_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [0]:
!pip install -U -q PyDrive
!pip install git+https://github.com/justadudewhohacks/image_augment.py
!pip install git+https://github.com/justadudewhohacks/colabsnippets

## mxnet

In [0]:
!nvcc --version
!pip install mxnet-cu100

## pytorch

In [0]:
!pip install torch torchvision

# Download Data

In [0]:
from colabsnippets.DataDownloader import DataDownloader

data_downloader = DataDownloader(data_dir = './data')

data_downloader.download_data({
	"celeba" : [
    #test data starts at shard 18
    { "images": "1C19zO2dlQz4ShGFJH_zEi3ToDOH-Iomp" },
    { "images": "1S5IIoHbNjv3p7PkJ3ItoAeQJODBX597-" },
    { "images": "1e6TUSjz7Zv6LmDP69Yc3lRIufeK3FuAR" }
	]
}, [])
data_downloader.drive.CreateFile({ 'id': '1rujtZzpWX5DP7E7HaxgqOPpk0cRb0ajD' }).GetContentFile('./data/celeba/landmarks.json')
data_downloader.download_data({
	"WIDER" : [
    { "images": "1JHmXqGPngDCbM56eYPeqsaCgJC4vgL4m", "boxes": "1Hd2i-6dnaWIriFK4Hj0CLZnfGtKcKj9L" }
	]
}, ['boxes'])
data_downloader.download_data({
	"FDDB" : [
    { "images": "1C8RpAZYg5nsPCAOLESGFwKxEaHx5V3Ag", "boxes": "1ACZPuSB7j0c0_hDIBL_MSW4rrzPhC1ab" }
	]
}, ['boxes'])

print('done!')

# DataLoader

In [0]:
import cv2
import math
import json
import random
import time
import types
import os
import numpy as np
import tensorflow as tf
from augment import ImageAugmentor, augment
from augment.augment import abs_coords
from colabsnippets.utils import load_json
from colabsnippets import BatchLoader

'''
--------------------------------------------------------------------------------

Data Loader

--------------------------------------------------------------------------------
'''
celeba_landmarks_by_file = load_json('./data/celeba/landmarks.json')

def min_bbox_from_pts(pts):
  min_x, min_y, max_x, max_y = 1.0, 1.0, 0, 0
  for pt in pts:
    x, y = pt
    min_x = x if x < min_x else min_x
    min_y = y if y < min_y else min_y
    max_x = max_x if x < max_x else x
    max_y = max_y if y < max_y else y

  return [min_x, min_y, max_x, max_y]
  
def json_boxes_to_array(boxes):
  out_boxes = []
  for box in boxes:
    x, y, w, h = box['x'], box['y'], box['width'], box['height']
    out_box = (x, y, w, h)     
    if w <= 0 or h <= 0:
      raise Exception("box has invalid width or height: {}".format(out_box))   
    for val in out_box:
      if val < -0.5 or val > 1.5:
        raise Exception("box is probably not a valid relative box: {}".format(out_box))
    out_boxes.append(out_box)
  return out_boxes
 
def extract_data_labels(data):
  db = data['db']
  img_file = data['file']
  if db == 'celeba':
    landmarks = celeba_landmarks_by_file[img_file]
    x, y, max_x, max_y = min_bbox_from_pts(landmarks)
    w, h = max_x - x, max_y - y
    padding = 1.5

    x = x - (0.5 * padding * w)
    y = y - (0.5 * padding * h)
    w = w + (padding * w)
    h = h + (padding * h)

    return [(x, y, w, h)]
  if db == 'WIDER' or db == 'FDDB':
    boxes_file = img_file.replace('.jpg', '.json')
    boxes_dir = "boxes-shard{}".format(data['shard']) if 'shard' in data else 'boxes'
    boxes_path = "./data/{}/{}/{}".format(db, boxes_dir, boxes_file)
    boxes = load_json(boxes_path)
    return json_boxes_to_array(boxes)
  
  raise Exception("extract_data_labels - unknown db '{}'".format(db))
    
    
def resolve_image_path(data):
  db = data['db']
  img_file = data['file']
  img_dir = "images-shard{}".format(data['shard']) if 'shard' in data else 'images'
  img_path = "./data/{}/{}/{}".format(db, img_dir, img_file)
  return img_path

class DataLoader(BatchLoader):
  def __init__(self, data):  
    BatchLoader.__init__(
      self, 
      data if type(data) is types.FunctionType else lambda: data, 
      resolve_image_path, 
      extract_data_labels,
      start_epoch = None, 
      is_test = True
    )
      
  def load_image_and_labels_batch(self, datas, image_size):
    batch_x, batch_y = [], []
    for data in datas:
      image = self.load_image(data)
      boxes = self.extract_data_labels(data)
      image, boxes = augment(image, boxes = boxes, pad_to_square = True, resize = image_size)
      batch_x.append(image)
      batch_y.append(boxes)
        
    return batch_x, batch_y
      

# Retina Face

In [0]:
!git clone https://github.com/deepinsight/insightface
!cd ./insightface/RetinaFace && make
!wget https://www.dropbox.com/s/53ftnlarhyrpkg2/retinaface-R50.zip -O retinaface-R50.zip
!unzip retinaface-R50.zip

In [0]:
import cv2
import sys
import numpy as np
import datetime
import os
import glob
import sys
sys.path.append('./insightface/RetinaFace')
from insightface.RetinaFace.retinaface import RetinaFace

def create_retina_face(scales = [1.0], do_flip = False):
  detector = RetinaFace('./R50', 0, 0, 'net3')
  def detect(img, min_score):
    faces, landmarks = detector.detect(img, min_score, scales=scales, do_flip=do_flip)
    out_faces = []
    img_size = img.shape[0]
    if faces is not None:
      for face in faces:
        x0, y0, x1, y1 = face.astype(np.int)[0:4] / img_size
        out_faces.append([x0, y0, x1-x0, y1-y0])
    return out_faces
  return detect

# Test

## Forward

In [0]:
from time import time

dbs = ['celeba', 'fddb', 'wider']
min_score = 0.5
min_image_size = 128
max_image_size =  608
detect = create_retina_face(scales = [1.0], do_flip = False)
  
for db in dbs:
  for image_size in range(min_image_size, max_image_size + 1, 32):
    print(db, image_size)
    out_filename = "retina_scales_1.0_flip_no_{}_{}".format(db, image_size)

    test_data = load_json("./{}_testData.json".format(db))
    data_loader = DataLoader(test_data)
    get_next_batch = lambda : data_loader.next_batch(1, image_size)

    next_batch = get_next_batch()
    results = []
    avg_time = 0
    while (next_batch != None):
      batch_x, batch_gt_boxes = next_batch
      gt_boxes = batch_gt_boxes[0]
      ts = time()
      boxes = detect(batch_x[0], min_score)
      avg_time += ((time() - ts) * 1000)
      results.append((boxes, gt_boxes))
      next_batch = get_next_batch()
    avg_time = avg_time / len(results)
    print(avg_time)
    np.save(out_filename, results, allow_pickle = True)

## Test

In [0]:
from colabsnippets.face_detection import calculate_iou

min_score = 0.5
image_size = 128
filename = "retina_celeba_{}.npy".format(image_size)

results = np.load(filename, allow_pickle = True)

#TODO NMS

true_positives = 0
false_positives = 0
total_boxes = 0
for res in results:
  pred_boxes, gt_boxes = res
  pred_boxes = np.array(pred_boxes) * image_size
  gt_boxes = np.array(gt_boxes) * image_size
  total_boxes += len(gt_boxes)


  for gt_box in gt_boxes:
    is_detected = False
    for pred_idx, pred_box in enumerate(pred_boxes):
      iou = calculate_iou(pred_box, gt_box)
      if iou > 0.5:
        is_detected = True
      else:
        false_positives += 1

    if is_detected:
      true_positives += 1

print(filename)
print("- true_positives: {}".format(true_positives))
print("- false_positives: {}".format(false_positives))
print("- true_positives ratio: {}".format(true_positives / total_boxes))
print("- false_positives ratio: {}".format(false_positives / total_boxes))
